In [2]:
import requests
import pandas as pd
import logging
from sqlalchemy import create_engine

# Set up logging
logging.basicConfig(level=logging.INFO)

# Create SQLite engine
engine = create_engine('sqlite:///ctgov.db')

# API setup
base_url = "https://clinicaltrials.gov/api/v2/studies"
params = {
    'filter.advanced': 'AREA[StartDate]RANGE[01/01/2024,MAX]',
    'pageSize': 100,
    'format': 'json',
}

# Initialize flag to control table creation
first_page = True

while True:
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        studies = data.get('studies', [])
        if not studies:
            break

        df = pd.json_normalize(studies)
        df = df.astype(str)

        # Optional: convert and select specific columns
        if 'protocolSection.statusModule.startDateStruct.date' in df.columns:
            df['startDate'] = pd.to_datetime(df['protocolSection.statusModule.startDateStruct.date'], errors='coerce')

        columns_to_keep = [
            'protocolSection.identificationModule.nctId',
            'protocolSection.statusModule.overallStatus',
            'protocolSection.statusModule.startDateStruct.date',
            'startDate',
        ]
        df = df[[col for col in columns_to_keep if col in df.columns]]

        # Append to SQLite DB
        df.to_sql('clinical_trials', engine, if_exists='replace' if first_page else 'append', index=False)
        first_page = False

        # Check for pagination
        nextPageToken = data.get('nextPageToken')
        if nextPageToken:
            params['pageToken'] = nextPageToken
        else:
            break
    else:
        logging.error(f"Failed to fetch data. Status code: {response.status_code}")
        break

logging.info("All data saved to SQLite.")


INFO:root:All data saved to SQLite.
